<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FSD_H2E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CASE1

In [1]:
import numpy as np

class VJEPA_WorldModel:
    """Simulates the 'Agent' part of H2E: Predicting the latent future."""
    def predict_future_latent(self, current_obs, action):
        # In a real V-JEPA, this would be a forward pass through a Transformer
        # Here we simulate the latent state: [x_pos, y_pos, velocity, ttc_to_object, light_status]
        # light_status: 0 = Green, 1 = Red

        new_velocity = action['accel'] * 0.5 + current_obs['velocity']
        new_dist = current_obs['dist_to_lead'] - (new_velocity * 0.1)

        # Latent representation of the future
        latent_future = {
            'velocity': new_velocity,
            'ttc': new_dist / (new_velocity + 1e-5),
            'light_state': current_obs['upcoming_light'],
            'lateral_force': abs(action['steering'] * new_velocity)
        }
        return latent_future

class RoadSafetyExpert:
    """The 'Expert' (E) in H2E: The symbolic governor that audits the Agent."""
    def __init__(self):
        self.LIMITS = {
            'max_v': 35.0,        # m/s (~126 km/h)
            'min_ttc': 1.5,       # Seconds (Time to Collision)
            'max_lat_g': 4.5,     # m/s^2 (Safety envelope)
            'stop_threshold': 0.2 # m/s
        }

    def audit(self, latent_future):
        violations = []

        # Rule 1: Physics/Safety - Collision Avoidance
        if latent_future['ttc'] < self.LIMITS['min_ttc']:
            violations.append(f"CRITICAL: Collision Risk (TTC: {latent_future['ttc']:.2f}s)")

        # Rule 2: Legal - Traffic Light Compliance
        if latent_future['light_state'] == 1 and latent_future['velocity'] > self.LIMITS['stop_threshold']:
            violations.append("LEGAL: Red light violation predicted")

        # Rule 3: Physics - Rollover/Skid protection
        if latent_future['lateral_force'] > self.LIMITS['max_lat_g']:
            violations.append("PHYSICS: Lateral G-force exceeds tire friction limit")

        return violations

class H2E_FSD_Controller:
    """The 'Human' (H) + Agent + Expert Integration."""
    def __init__(self):
        self.agent = VJEPA_WorldModel()
        self.expert = RoadSafetyExpert()

    def drive_step(self, human_intent, sensor_data):
        print(f"--- Processing Step: Intent {human_intent['mode']} ---")

        # 1. Agent proposes an action based on Human Intent (e.g., 'Target Speed')
        proposed_action = {
            'accel': 2.0 if sensor_data['velocity'] < human_intent['target_v'] else -1.0,
            'steering': 0.05 # slight curve
        }

        # 2. V-JEPA Predicts the 'Latent Future'
        future = self.agent.predict_future_latent(sensor_data, proposed_action)

        # 3. Expert Audits the Prediction
        violations = self.expert.audit(future)

        if not violations:
            print(f"H2E Status: [APPROVED] - Executing {proposed_action}")
            return proposed_action
        else:
            print(f"H2E Status: [REJECTED] - Safety Layer intervened.")
            for v in violations: print(f"  -> {v}")
            # Safe State Fallback
            return {'accel': -5.0, 'steering': 0.0} # Emergency Braking

# --- SIMULATION RUN ---
fsd_system = H2E_FSD_Controller()

# Scenario: Driving towards a Red Light
current_sensors = {
    'velocity': 10.0,
    'dist_to_lead': 15.0,
    'upcoming_light': 1 # RED
}

user_intent = {'mode': 'Normal', 'target_v': 15.0}

# Run one control loop cycle
final_command = fsd_system.drive_step(user_intent, current_sensors)
print(f"\nFinal Actuator Command: {final_command}")

--- Processing Step: Intent Normal ---
H2E Status: [REJECTED] - Safety Layer intervened.
  -> CRITICAL: Collision Risk (TTC: 1.26s)
  -> LEGAL: Red light violation predicted

Final Actuator Command: {'accel': -5.0, 'steering': 0.0}


## CASE2

In [2]:
import numpy as np

class VJEPA_WorldModel:
    """The 'Agent': Predicting latent trajectories for ego and external actors."""
    def predict_future_latent(self, current_obs, action, weather_condition='dry'):
        # Physics constants based on environment
        friction_coeff = 0.9 if weather_condition == 'dry' else 0.4

        new_velocity = action['accel'] * 0.5 + current_obs['velocity']
        new_dist = current_obs['dist_to_lead'] - (new_velocity * 0.1)

        # Latent representation including environmental variables
        latent_future = {
            'velocity': new_velocity,
            'ttc': new_dist / (new_velocity + 1e-5),
            'light_state': current_obs['upcoming_light'],
            'lateral_force': abs(action['steering'] * new_velocity),
            'friction_limit': friction_coeff * 9.81,
            'pedestrian_proximity': current_obs.get('pedestrian_dist', 100.0) - (new_velocity * 0.05)
        }
        return latent_future

class SovereignSafetyExpert:
    """The 'Expert' (E): Symbolic governor with dynamic environmental constraints."""
    def __init__(self):
        self.BASE_LIMITS = {
            'min_ttc': 1.8,           # Increased safety buffer
            'stop_threshold': 0.1,
            'pedestrian_buffer': 5.0  # Meters
        }

    def audit(self, latent_future):
        violations = []

        # Rule 1: Dynamic Physics - Friction/Hydroplaning Audit
        if latent_future['lateral_force'] > (latent_future['friction_limit'] * 0.8):
            violations.append(f"PHYSICS: Lateral force ({latent_future['lateral_force']:.2f}) exceeds safe friction ceiling.")

        # Rule 2: Multi-Agent Safety - Pedestrian Buffer
        if latent_future['pedestrian_proximity'] < self.BASE_LIMITS['pedestrian_buffer']:
            violations.append(f"SAFETY: Pedestrian proximity ({latent_future['pedestrian_proximity']:.2f}m) violation.")

        # Rule 3: Legal - Signal Compliance
        if latent_future['light_state'] == 1 and latent_future['velocity'] > self.BASE_LIMITS['stop_threshold']:
            violations.append("LEGAL: Red light violation predicted.")

        # Rule 4: Critical Distance
        if latent_future['ttc'] < self.BASE_LIMITS['min_ttc']:
            violations.append(f"CRITICAL: Collision Risk (TTC: {latent_future['ttc']:.2f}s).")

        return violations

class H2E_FSD_Controller:
    """The H2E Integration with Sovereign Audit Logs."""
    def __init__(self):
        self.agent = VJEPA_WorldModel()
        self.expert = SovereignSafetyExpert()
        self.audit_log = []

    def drive_step(self, human_intent, sensor_data, weather='rain'):
        print(f"--- H2E Cycle: {human_intent['mode']} Mode | Weather: {weather} ---")

        # 1. Agent Proposal
        proposed_action = {
            'accel': 2.5 if sensor_data['velocity'] < human_intent['target_v'] else -0.5,
            'steering': 0.4  # Sharp turn
        }

        # 2. V-JEPA Latent Prediction
        future = self.agent.predict_future_latent(sensor_data, proposed_action, weather)

        # 3. Expert Audit
        violations = self.expert.audit(future)

        if not violations:
            print(f"H2E Status: [APPROVED]")
            return proposed_action
        else:
            print(f"H2E Status: [REJECTED] - Expert Intervention")
            for v in violations:
                print(f"  [!] {v}")
                self.audit_log.append({'reason': v, 'state': sensor_data})

            # Safe State: Dynamic Braking
            return {'accel': -6.0, 'steering': 0.0}

# --- ENHANCED SIMULATION RUN ---
fsd_system = H2E_FSD_Controller()

# Scenario: Sharp turn in the rain with a nearby pedestrian
current_sensors = {
    'velocity': 12.0,
    'dist_to_lead': 40.0,
    'upcoming_light': 0, # Green
    'pedestrian_dist': 6.0
}

user_intent = {'mode': 'Performance', 'target_v': 20.0}

final_command = fsd_system.drive_step(user_intent, current_sensors, weather='rain')
print(f"\nFinal Actuator Command: {final_command}")

--- H2E Cycle: Performance Mode | Weather: rain ---
H2E Status: [REJECTED] - Expert Intervention
  [!] PHYSICS: Lateral force (5.30) exceeds safe friction ceiling.

Final Actuator Command: {'accel': -6.0, 'steering': 0.0}


## CASE3

In [4]:
import numpy as np

class VJEPA_WorldModel:
    """The 'Agent': Predicting latent futures for ego and surrounding environment."""
    def predict_future_latent(self, current_obs, action, weather_condition='dry'):
        friction_coeff = 0.9 if weather_condition == 'dry' else 0.4
        new_velocity = (action['accel'] * 0.5) + current_obs['velocity']

        # Latent representation including multi-agent and lane states
        latent_future = {
            'velocity': new_velocity,
            'ttc': (current_obs['dist_to_lead'] - (new_velocity * 0.1)) / (new_velocity + 1e-5),
            'light_state': current_obs['upcoming_light'],
            'lateral_force': abs(action['steering'] * new_velocity),
            'friction_limit': friction_coeff * 9.81,
            # Pass-through sensor data into the latent future for the Expert to audit
            'pedestrian_proximity': current_obs.get('pedestrian_dist', 100.0) - (new_velocity * 0.1),
            'oncoming_traffic': current_obs.get('oncoming_traffic', False),
            'lane_violation': 1.0 if abs(action['steering']) > 0.5 else 0.0
        }
        return latent_future

class SovereignSafetyExpert:
    """The 'Expert' (E): Solving the 'Double-Bind' via Hierarchical Governance."""
    def __init__(self):
        self.LIMITS = {'min_ttc': 1.8, 'pedestrian_buffer': 5.0}

    def audit(self, latent_future):
        violations = []

        # 1. Detect Potential Pedestrian Collision
        ped_risk = latent_future['pedestrian_proximity'] < self.LIMITS['pedestrian_buffer']

        # 2. Detect Legal Boundary Breach (Lane Crossing)
        lane_risk = latent_future['lane_violation'] > 0.5

        # 3. Apply H2E Hierarchical Logic
        # PRIORITY: Life Safety > Traffic Law
        if ped_risk:
            if not latent_future['oncoming_traffic']:
                # AUTHORIZED EXCEPTION: Save life if path is clear
                return ["ADVISORY: Authorized Lane Violation (Evasive Maneuver - Life Safety)"]
            else:
                violations.append("CRITICAL: Pedestrian risk high BUT oncoming lane blocked.")

        if lane_risk and not ped_risk:
            violations.append("LEGAL: Unauthorized lane crossing.")

        return violations

class H2E_FSD_Controller:
    def __init__(self):
        self.agent = VJEPA_WorldModel()
        self.expert = SovereignSafetyExpert()

    def drive_step(self, human_intent, sensor_data):
        print(f"--- H2E Decision Loop: {human_intent['mode']} ---")

        # Agent proposes a sharp swerve to avoid the child
        proposed_action = {'accel': -2.0, 'steering': 0.8}

        future = self.agent.predict_future_latent(sensor_data, proposed_action)
        audit_results = self.expert.audit(future)

        if any("ADVISORY" in msg for msg in audit_results):
            print(f"H2E Status: [APPROVED WITH EXCEPTION]")
            for msg in audit_results: print(f"  {msg}")
            return proposed_action
        elif audit_results:
            print(f"H2E Status: [REJECTED]")
            for v in audit_results: print(f"  [!] {v}")
            return {'accel': -8.0, 'steering': 0.0} # Maximum Braking

        return proposed_action

# --- TEST CASE: Evasive Maneuver ---
fsd_system = H2E_FSD_Controller()
complex_sensors = {
    'velocity': 12.0,
    'dist_to_lead': 60.0,
    'upcoming_light': 0,
    'pedestrian_dist': 4.5,   # Child in road
    'oncoming_traffic': False # Safe to swerve
}

final_cmd = fsd_system.drive_step({'mode': 'Safety'}, complex_sensors)
print(f"\nFinal Command: {final_cmd}")

--- H2E Decision Loop: Safety ---
H2E Status: [APPROVED WITH EXCEPTION]
  ADVISORY: Authorized Lane Violation (Evasive Maneuver - Life Safety)

Final Command: {'accel': -2.0, 'steering': 0.8}
